In [2]:
import sys
sys.path.append('../../')

In [19]:
from dataclasses import dataclass
import itertools
import requests
import pandas as pd

from utils.subgraph_utils.constants import SUBGRAPH_API

In [4]:
def get_pool_data(api):

    query = """
    {
        platforms {
            pools {
                coins
                coinDecimals
                address
            }
        }
    }
    """
    r = requests.post(api, json={"query": query})
    data = dict(r.json())
    pool_data = data["data"]["platforms"][0]["pools"]

    return pool_data

In [8]:
network_name = 'Mainnet'

In [9]:
data = get_pool_data(SUBGRAPH_API[network_name])
data[:2]

[{'coins': ['0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
   '0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'],
  'coinDecimals': ['18', '18'],
  'address': '0x0043fcb34e7470130fde28198571dee092c70bd7'},
 {'coins': ['0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415',
   '0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050'],
  'coinDecimals': ['18', '18'],
  'address': '0x01fe650ef2f8e2982295489ae6adc1413bf6011f'}]

In [13]:
@dataclass
class Coin:
    """Class for keeping track of an item in inventory."""
    address: str
    network: str
    decimals: int

    def convert_to_float(self, input_amount: int) -> float:
        return input_amount / 10 ** self.decimals

In [37]:
coin_map = {}  # pool_a: coin pair; a = 1, 2, 3, ... depending on number of coins in the pool.
for pool in data:
    pool_address = pool['address']

    coins = []    
    for idx, coin in enumerate(pool['coins']):
        coin_dataclass = Coin(address=coin, network=network_name, decimals=int(pool['coinDecimals'][idx]))
        coins.append(coin_dataclass)
    
    coin_permutations = list(itertools.permutations(coins))
    
    for coin_permutation in coin_permutations:

        num_pool_entries = len([1 for i in coin_map.keys() if i.startswith(pool_address)])
        pool_key = pool_address + '_' + str(num_pool_entries + 1)
        coin_map[pool_key] = coin_permutation

coin_map[list(coin_map.keys())[0]]

(Coin(address='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f', network='Mainnet', decimals=18),
 Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490', network='Mainnet', decimals=18))